## Langchain Expression Language Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

In [5]:
from dotenv import load_dotenv

load_dotenv('./../.env')

True

### Sequential LCEL Chain

In [11]:
from langchain_ollama import ChatOllama

from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate
                                        )

from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(base_url=base_url, model=model)

system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')

messages = [system, question]

template = ChatPromptTemplate(messages)

In [12]:
# create chain
# input | model | output (str output)
chain = template | llm | StrOutputParser()

In [14]:
response = chain.invoke({'school': 'elementary', 'topics': 'sun', 'points': 5})
print(response)

Here's what I know about the sun:

• The sun is a huge ball of hot, glowing gas.
• It's about 93 million miles away from Earth.
• The sun makes all the light and heat that we need for life on our planet.
• Without the sun, we would be frozen in place, unable to grow or move around.
• We can see only one side of the sun because it reflects sunlight back towards us, making it look bright and round.


### Chaining Runnables (Chain Multiple Runnables)

- We can even combine this chain with more runnables to create another chain.
- Let's see how easy our generated output is?

In [20]:
analysis_prompt = ChatPromptTemplate.from_template('analyze the following fact: {fact}\n\nHow easy is this fact to understand? Tell me in 10 words.')
analysis_prompt

fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'fact': response})
print(output)

The fact is relatively simple and easy to comprehend with basic knowledge.


In [24]:
composed_chain = {"fact": chain} | analysis_prompt | llm | StrOutputParser()

output = composed_chain.invoke({'school': 'ph.d', 'topics': 'sun', 'points': 5})
print(output)

The concept of a massive star's immense size and nuclear reaction complexity is hard to grasp.


### Parallel LCEL Chain
- Parallel chains are used to run multiple runnables in parallel.
- The final return value is a dict with the results of each value under its appropriate key.

In [28]:
llm = ChatOllama(base_url=base_url, model=model)

system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')

messages = [system, question]

template = ChatPromptTemplate(messages)

fact_chain = template | llm | StrOutputParser()

In [29]:
question = HumanMessagePromptTemplate.from_template('write a poem about the {topics} in {sentences} sentences')

messages = [system, question]

template = ChatPromptTemplate(messages)

poem_chain = template | llm | StrOutputParser()

In [33]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(fact = fact_chain, poem=poem_chain)

response = chain.invoke({'school': 'elementary', 'topics': 'sun', 'points': 5, 'sentences': 5})
print(response)

{'fact': "Here's what I know about the sun:\n\n1. **The Sun is a Star**: It's a massive ball of hot, glowing gas that lives at the center of our solar system.\n\n2. **It's Really Hot**: The surface temperature of the sun is about 5,500 degrees Celsius. That's really, really hot!\n\n3. **The Sun is Huge**: Our sun is about 109 times bigger than Earth and has a diameter of about 1.4 million kilometers.\n\n4. **It Makes Us Warm**: The sun's rays help give us warmth on Earth by heating up the planet.\n\n5. **We're Not Alone**: There are billions of other stars in our galaxy, and some of them might be similar to the sun in size and temperature.", 'poem': 'The sun rises high in the sky,\nBringing light to the world outside.\nIts warm rays shine down on our face,\nWarming us up with a gentle pace,\nA beautiful sight, a lovely hue.'}


In [36]:
response.keys()

print(response['fact'])

print("\n\n")
print(response['poem'])

Here's what I know about the sun:

1. **The Sun is a Star**: It's a massive ball of hot, glowing gas that lives at the center of our solar system.

2. **It's Really Hot**: The surface temperature of the sun is about 5,500 degrees Celsius. That's really, really hot!

3. **The Sun is Huge**: Our sun is about 109 times bigger than Earth and has a diameter of about 1.4 million kilometers.

4. **It Makes Us Warm**: The sun's rays help give us warmth on Earth by heating up the planet.

5. **We're Not Alone**: There are billions of other stars in our galaxy, and some of them might be similar to the sun in size and temperature.



The sun rises high in the sky,
Bringing light to the world outside.
Its warm rays shine down on our face,
Warming us up with a gentle pace,
A beautiful sight, a lovely hue.


### Chain Router
- The router chain is used to route the output of a previous runnable to the next runnable based on the output of the previous runnable.

In [69]:
prompt = """Given the user review below, classify it as either being about `Positive` or `Negative`.
            Do not respond with more than one word.

            Question: {question}
            Classification:"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()

question = "Thank you so much for providing such a great platform for learning."
chain.invoke({"question": question})

'Positive'

In [87]:
positive_prompt = """
                You are expert in writing reply for positive reviews.
                You need to encourage the user to share their experience on social media.
                Question: {question}
                Answer:"""
positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()


negative_prompt = """
                You are expert in writing reply for negative reviews.
                You need first to apologize for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following Email:'udemy@kgptalkie.com'.
                Question: {question}
                Answer:"""
negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()




In [88]:
def route(info):
    if "positive" in info["topic"].lower():
        return positive_chain
    else:
        return negative_chain

In [89]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

In [90]:
question

'I am not happy with the course content. I want my refund.'

In [91]:
response = full_chain.invoke({"question": question})
print(response)

Here's a sample reply:

Dear valued customer,

We apologize for any inconvenience or frustration caused by the course content that didn't meet your expectations. We understand that our courses are designed to provide valuable learning experiences, and it's disappointing when this doesn't align with your goals.

We want to assure you that we take all concerns seriously and would like to help resolve this matter. Unfortunately, as a gesture of goodwill, we cannot offer a refund at this time. However, we value your feedback and would appreciate any additional information that might help us improve our course content in the future.

If you're willing, please could you contact us at udemy@kgptalkie.com so we can discuss further how we can better address your concerns? Your input will play a crucial role in shaping the next version of our courses. We're committed to providing high-quality educational resources and appreciate your patience and understanding during this time.

Thank you for ch

In [92]:
question = "I am not happy with the course content. I want my refund."
response = full_chain.invoke({"question": question})
print(response)

Here's a potential response:

"Dear [User's Name],

I'm so sorry to hear that you're not satisfied with the course content and are looking for a refund. I understand how frustrating it can be when we don't meet our expectations.

As per our policies, refunds will be processed within 7-10 business days after receiving your request. However, please note that there might be some delay in processing due to various reasons such as administrative tasks or external factors.

I'd like to suggest an alternative solution. We're committed to providing you with a high-quality learning experience. I'd love to discuss this further with you and see if there's anything we can do to rectify the situation. Please feel free to share your concerns and suggestions via our email address: udyetalkie@kgptalkie.com.

Your feedback is invaluable in helping us improve, and I'm confident that together, we can find a resolution that meets your expectations.

Thank you for choosing Udemy and talkie, and I look forw

### Make Custom Chain Runnables with RunnablePassthrough and RunnableLambda
- This is useful for formatting or when you need functionality not provided by other LangChain components, and custom functions used as Runnables are called RunnableLambdas.



In [104]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())


prompt = ChatPromptTemplate.from_template("what is {in1} + {in2}")

chain1 = prompt | llm

chain = (
    prompt
    | llm
    | StrOutputParser()
    |{
        "char_counts": RunnableLambda(char_counts),
        "word_counts": RunnableLambda(word_counts),
        "output": RunnablePassthrough()
    }
)

chain.invoke({"in1": "bar", "in2": "gah"})

{'char_counts': 899,
 'word_counts': 137,
 'output': 'Bar + Gaze are a New York-based indie folk duo consisting of Alex Hall and Gabe Nichols. They create music that blends elements of folk, rock, and electronic music with introspective and emotionally charged lyrics.\n\nTheir sound is characterized by Hall\'s soaring vocals, Nichols\' atmospheric instrumentation, and the way they weave together storytelling and poetic imagery to convey complex emotions and ideas. The duo\'s music often explores themes of love, loss, identity, and social commentary, all set to a backdrop of haunting melodies and textures.\n\nBar + Gaze have released several EPs and albums throughout their career, including "Bar" (2014) and "Gaze" (2015). Their music has been praised for its originality, sensitivity, and emotional depth. If you\'re looking for artists who blend folk and electronic elements with introspective songwriting, Bar + Gaze is definitely worth checking out.'}

### Custom Chain using `@chain` decorator

In [107]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain


@chain
def custom_chain(params):
    return {
        "facts": fact_chain.invoke(params),
        "poem": poem_chain.invoke(params)
    }


params = {'school': 'elementary', 'topics': 'sun', 'points': 5, 'sentences': 5}
custom_chain.invoke(params)

{'facts': "Here's what you need to know about the sun:\n\n1. The sun is a star that gives us light and warmth.\n2. It's about 93 million miles away from Earth.\n3. The sun makes up 99% of our solar system's mass.\n4. It takes about 8 minutes and 20 seconds for sunlight to reach Earth.\n5. The surface temperature of the sun is about 5,500 degrees Celsius.",
 'poem': 'The sun shines bright in the morning sky,\nBringing light to the world, as it passes by.\nIt rises high and sets low with a grin,\nWarming our skin and making everything win.\nA beautiful sight that we all can see.'}

In [105]:
fact_chain

ChatPromptTemplate(input_variables=['points', 'school', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topics'], input_types={}, partial_variables={}, template='tell me about the {topics} in {points} points'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()

In [106]:
poem_chain

ChatPromptTemplate(input_variables=['school', 'sentences', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['sentences', 'topics'], input_types={}, partial_variables={}, template='write a poem about the {topics} in {sentences} sentences'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()